# Demo 2 : Data Modeling, Creating a Table with PostgreSQL

<img src="images/postgresql-logo.png" width="250" height="250">

configuring Postgres, creating users, and creating databases using the psql utility
https://www.codementor.io/@engineerapart/getting-started-with-postgresql-on-mac-osx-are8jcopb

psycopg2 https://pynative.com/python-postgresql-tutorial/

# PostgreSQL and AutoCommits

## Walk through the basics of PostgreSQL autocommits

In [1]:
## import postgreSQL adapter for the Python
import psycopg2

/Users/kholed/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Create a connection to the database
1. Connect to the local instance of PostgreSQL (*127.0.0.1*)
2. Use the database/schema from the instance. 
3. The connection reaches out to the database (*dataenginneering*) and use the correct privilages to connect to the database (*user = postgres and password = admin*).

In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=dataengineering user=postgres password=admin")

### Use the connection to get a cursor that will be used to execute queries.

https://www.postgresql.org/docs/9.2/plpgsql-cursors.html

In [6]:
cur = conn.cursor()

### Create a database to work in

In [9]:
cur.execute("select * from test2")

ProgrammingError: relation "test2" does not exist
LINE 1: select * from test2
                      ^


### Error occurs, but it was to be expected because table has not been created as yet. To fix the error, create the table.¶

In [10]:
cur.execute("CREATE TABLE test2 (col1 int, col2 int, col3 int);")

InternalError: current transaction is aborted, commands ignored until end of transaction block


### Error indicates we cannot execute this query. Since we have not committed the transaction and had an error in the transaction block, we are blocked until we restart the connection.

In [11]:
conn = psycopg2.connect("host=127.0.0.1 dbname=dataengineering user=postgres password=admin")
cur = conn.cursor()

In our exercises instead of worrying about commiting each transaction or getting a strange error when we hit something unexpected, let's set autocommit to true. **This says after each call during the session commit that one action and do not hold open the transaction for any other actions. One action = one transaction.**

In this demo we will use automatic commit so each action is commited without having to call `conn.commit()` after each command. **The ability to rollback and commit transactions are a feature of Relational Databases.**

In [12]:
conn.set_session(autocommit=True)

In [13]:
cur.execute("select * from test2")

ProgrammingError: relation "test2" does not exist
LINE 1: select * from test2
                      ^


In [14]:
cur.execute("CREATE TABLE test2 (col1 int, col2 int, col3 int);")

### Once autocommit is set to true, we execute this code successfully. There were no issues with transaction blocks and we did not need to restart our connection. 

In [15]:
cur.execute("select * from test2")

In [17]:
cur.execute("select count(*) from test2")
print(cur.fetchall())

[(0,)]


# Creating a Table with PostgreSQL

## Walk through the basics of PostgreSQL:
<br><li>Creating a table <li>Inserting rows of data, <li>Running a simple SQL query to validate the information. 

### Typically, we would use a python wrapper called *psycopg2* to run the PostgreSQL queries. This library should be preinstalled but in the future to install this library, run the following command in the notebook to install locally: 
`!pip3 install --user psycopg2`
#### More documentation can be found here: http://initd.org/psycopg/ 

#### Import the library 
Note: An error might popup after this command has executed. Read it carefully before proceeding.

In [3]:
import psycopg2

/Users/kholed/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


### Create a connection to the database
1. Connect to the local instance of PostgreSQL (*127.0.0.1*)
2. Use the database/schema from the instance. 
3. The connection reaches out to the database (*dataenginneering*) and use the correct privilages to connect to the database (*user = postgres and password = admin*).

### Note 1: This block of code will be standard in all notebooks. 
### Note 2: Adding the `try except` will make sure errors are caught and understood

In [4]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=dataengineering user=postgres password=admin")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

### Use the connection to get a cursor that can be used to execute queries.

In [5]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

### Use automactic commit so that each action is commited without having to call conn.commit() after each command. The ability to rollback and commit transactions is a feature of Relational Databases. 

In [6]:
conn.set_session(autocommit=True)

### Test the Connection and Error Handling Code
The try-except block should handle the error: We are trying to do a select * on a table but the table has not been created yet.

In [7]:
try: 
    cur.execute("select * from quran.surah")
except psycopg2.Error as e:
    print(e)

relation "quran.surah" does not exist
LINE 1: select * from quran.surah
                      ^



### Create a database to work in 

In [8]:
try: 
    cur.execute("create database quran")
except psycopg2.Error as e:
    print(e)

database "quran" already exists



### Close our connection to the default database, reconnect to the Quran database, and get a new cursor.

In [9]:
try: 
    conn.close()
except psycopg2.Error as e:
    print(e)
  
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=quran user=postgres password=admin")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

conn.set_session(autocommit=True)

### We will create AL-QURAN, each surah has a lot of information we could add to the AL-QURAN database. We will design english translation by create quran_index, sura, aya, text

`Table Name: English Saheeh International 
column 1: Quran Index
column 2: Surah
column 3: Ayah
column 4: Text Translation`

### Translate this information into a Create Table Statement. 

Review this document on PostgreSQL datatypes: https://www.postgresql.org/docs/9.5/datatype.html


In [10]:
try: 
    cur.execute("CREATE TABLE IF NOT EXISTS en_sahih (index int, surah int, ayah int, text varchar);")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

### No error was found, but lets check to ensure our table was created.  `select count(*)` which should return 0 as no rows have been inserted in the table.

In [11]:
try: 
    cur.execute("select count(*) from en_sahih")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
print(cur.fetchall())

[(0,)]


### Insert a row 

In [12]:
## 1
try: 
    cur.execute("INSERT INTO en_sahih (index, surah, ayah, text) \
                 VALUES (%s, %s, %s, %s)", \
                 (1, 1, 1, 'In the name of Allah, the Entirely Merciful, the Especially Merciful.'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

### Insert multiple rows

In [13]:
## 2
try: 
    cur.execute("INSERT INTO en_sahih (index, surah, ayah, text) \
                 VALUES (%s, %s, %s, %s)", \
                 (2, 1, 2, '[All] praise is [due] to Allah, Lord of the worlds -'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

## 3
try: 
    cur.execute("INSERT INTO en_sahih (index, surah, ayah, text) \
                 VALUES (%s, %s, %s, %s)", \
                 (3, 1, 3, 'The Entirely Merciful, the Especially Merciful,'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

## 4
try: 
    cur.execute("INSERT INTO en_sahih (index, surah, ayah, text) \
                 VALUES (%s, %s, %s, %s)", \
                 (4, 1, 4, 'Sovereign of the Day of Recompense.'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

## 5
try: 
    cur.execute("INSERT INTO en_sahih (index, surah, ayah, text) \
                 VALUES (%s, %s, %s, %s)", \
                 (5, 1, 5, 'It is You we worship and You we ask for help.'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
## 6
try: 
    cur.execute("INSERT INTO en_sahih (index, surah, ayah, text) \
                 VALUES (%s, %s, %s, %s)", \
                 (6, 1, 6, 'Guide us to the straight path -'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

## 7
try: 
    cur.execute("INSERT INTO en_sahih (index, surah, ayah, text) \
                 VALUES (%s, %s, %s, %s)", \
                 (7, 1, 7, 'The path of those upon whom You have bestowed favor, not of those who have evoked [Your] anger or of those who are astray.'))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

### Validate your data was inserted into the table. 
The while loop is used for printing the results. If executing queries in the Postgres shell, this would not be required.

### Note: If you run the insert statement code more than once, you will see duplicates of your data. PostgreSQL allows for duplicates.

In [29]:
try: 
    cur.execute("SELECT * FROM en_sahih;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 1, 1, 'In the name of Allah, the Entirely Merciful, the Especially Merciful.')
(2, 1, 2, '[All] praise is [due] to Allah, Lord of the worlds -')
(3, 1, 3, 'The Entirely Merciful, the Especially Merciful,')
(4, 1, 4, 'Sovereign of the Day of Recompense.')
(5, 1, 5, 'It is You we worship and You we ask for help.')
(6, 1, 6, 'Guide us to the straight path -')
(7, 1, 7, 'The path of those upon whom You have bestowed favor, not of those who have evoked [Your] anger or of those who are astray.')


### Drop the table to avoid duplicates and clean up

In [31]:
try: 
    cur.execute("DROP table en_sahih")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

Error: Dropping table
table "en_sahih" does not exist



###  Close the cursor and connection. 

In [32]:
cur.close()
conn.close()